# Exercise02 : リモート CPU VM でのトレーニング

前回の演習([Exercise01](./exercise01_train_on_local.ipynb))を CPU を利用したリモート仮想マシンで実行してみましょう。<br>
また、お好きな Docker イメージでリモートトレーニングを実行することも可能です。


## トレーニングスクリプトをファイルとして保存する (train.py)

``scirpt`` ディレクトリを作成し、Python スクリプトを ``./script/train.py`` として保存します。

In [ ]:
import os
script_folder = './scripts'
os.makedirs(script_folder, exist_ok=True)

In [ ]:
%%writefile scripts/train.py
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license.

import argparse, os, json
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.sklearn

import matplotlib
matplotlib.use('Agg')

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()
    # add arguments
    parser.add_argument("--input_data", type=str, help="input data")
    parser.add_argument("--output_dir", type=str, help="output dir", default="./outputs")
    # parse args
    args = parser.parse_args()

    return args

# define functions
def main(args):
        
    lines = [
        f"Training data path: {args.input_data}",
        f"output dir path: {args.output_dir}"
    ]
    for line in lines:
        print(line)

    diabetes_data = np.loadtxt(args.input_data, delimiter=',',skiprows=1)
    X=diabetes_data[:,:-1]
    y=diabetes_data[:,-1]
    #columns = ['age', 'gender', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    with mlflow.start_run():

        run_id = mlflow.active_run().info.run_id
        mlflow.autolog(log_models=False, exclusive=True)
        print('run_id = ', run_id)

        mlflow.log_metric("Training samples", len(X_train))
        mlflow.log_metric("Test samples", len(X_test))

        # Log the algorithm parameter alpha to the run
        mlflow.log_metric('alpha', 0.03)
        # Create, fit, and test the scikit-learn Ridge regression model
        regression_model = Ridge(alpha=0.03)
        regression_model.fit(X_train, y_train)
        preds = regression_model.predict(X_test)

        # Log mean squared error
        mse = mean_squared_error(y_test, preds)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, preds)

        mlflow.log_metric('mse', mse)
        mlflow.log_metric('rmse', rmse)
        mlflow.log_metric('R2', r2)

        # Plot actuals vs predictions and save the plot within the run
        plt.figure(figsize=(10, 7))

        #scatterplot of y_test and preds
        plt.scatter(y_test, preds) 
        plt.plot(y_test, y_test, color='r')

        plt.title('Actual VS Predicted Values (Test set)') 
        plt.xlabel('Actual Values') 
        plt.ylabel('Predicted Values')

        plt.savefig('actuals_vs_predictions.png')
        mlflow.log_artifact("actuals_vs_predictions.png")

        # Finally save the model to the outputs directory for capture
        os.makedirs(os.path.join(args.output_dir, 'models'), exist_ok=True)
        mlflow.sklearn.save_model(regression_model, os.path.join(args.output_dir, 'models'))

        metric = {}
        metric['run_id'] = run_id
        metric['RMSE'] = rmse
        metric['R2'] = r2
        print(metric)

        with open(os.path.join(args.output_dir, 'metric.json'), "w") as outfile:
            json.dump(metric, outfile)

        mlflow.log_artifacts(args.output_dir)

# run script
if __name__ == "__main__":
    # parse args
    args = parse_args()

    # run main function
    main(args)

## リモート仮想マシンでのトレーニング

それでは、AML と連携して、リモート仮想マシンのトレーニングを自動化する方法を紹介します。

### Step 1 : 新しいコンピューティング クラスターを作成する

開始する前に、以下を確認してください。

- AML ワークスペースが存在する場所で、以下のサイズ（以下のスクリプトでは、Standard_D2_v2）がサポートされていることを確認します。
- Azure サブスクリプションに ML CPU VM 用のクォータがあること。もしない場合は、Azure ポータルから割り当てを申請してください。

min-instances に 0 を指定すると、ノードがアクティブでない場合に終了させることができます。(コスト削減ができます)。<br>
既存の仮想マシンをコンピュート ターゲットとしてアタッチすることも可能です（独自のコンピュートリソースを持ち込む）。

In [ ]:
!az ml compute create --name demo-cpucluster1 \
  --type amlcompute \
  --min-instances 0 \
  --max-instances 1 \
  --size Standard_D2_v2 \
  --idle-time-before-scale-down 1800

ML Studio の左メニュー「コンピューティング」をクリックすると実際にコンピューティング クラスターが登録されていることが確認できます。

### Step 2 : 環境の作成

ここでは、スクリプトを実行するための Docker 環境を新規に作成します。初回は、以下のような設定で、自前の conatiner イメージを生成します。(ただし、一度生成した環境を登録しておけば、次回の実行時に再利用することで高速化できます。

この例では、手動で環境を作成していますが、既存の環境（curated environmentと呼びます）を利用することもできます。


まず conda の dependancies yaml を作成し、04_conda_pydata.yml として保存します。


In [ ]:
script_folder = './environments'
os.makedirs(script_folder, exist_ok=True)

モジュールのバージョン違いによるエラーなどの実行環境に起因する問題は、正常動作している開発環境と同じ設定にすることで軽減できます。

In [ ]:
%%writefile environments/02_diabetes-env.yml
channels:
  - defaults
  - anaconda
  - conda-forge
dependencies:
  - python=3.8.5
  - pip
  - pip:
      - azureml-defaults
      - azureml-mlflow==1.41.0
      - scikit-learn==1.0.2
      - pandas==1.1.5
      - joblib==1.0.0
      - matplotlib==3.3.3

カスタム環境（diabetes-env という名前）を上記の conda の設定で AML に登録します。

In [ ]:
%%writefile 02_env_register.yml
$schema: https://azuremlschemas.azureedge.net/latest/environment.schema.json
name: diabetes-env
image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04
version: 1
conda_file: ./environments/02_diabetes-env.yml
description: diabetes env

In [ ]:
!az ml environment create --file 02_env_register.yml

ML Studio の左メニュー「環境」をクリックすると実際にカスタム環境が登録されていることが確認できます。

### Step 3 : トレーニングジョブの送信

上記の compute と環境を用いてトレーニングジョブを送信します。
この例では、diabetes_data_oh4ml_350records という登録済みのデータ アセットをコンピュート ターゲットにマウントして使用します。(データの準備は "Exercise02 : Prepare Data "を実行してください。)
 AMLでデータ アセットを利用するには、以下のように　`azureml:{DATA_NAME}:{DATA_VERSION}` または `azureml:{DATA_NAME}@latest `に最新版のアセットを設定してください。

> Note : 注意：初回実行時は、ベースイメージの取得、新しいイメージ（カスタム環境）の生成、クラスタ内のノードの起動、スクリプトの実行を行うため、10~15 分の時間がかかります。組み込みのキュレーション環境を使用することで、高速化することができます。

In [ ]:
%%writefile 02_train_on_remote.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json
experiment_name: train-diabetes-oh4ml
description: train diabetes oh4ml
type: command
code: ./scripts
command: >-
  python train.py 
  --input_data ${{inputs.diabetes_data}}
  --output_dir ${{outputs.outputs}}
environment: azureml:diabetes-env@latest
inputs:
  diabetes_data:
    type: uri_file
    path: azureml:diabetes_data_oh4ml_350records@latest
  mlflow_exp_name: train-diabetes-oh4ml
outputs:
  outputs:
compute: azureml:demo-cpucluster1

では、AML CLI v2 でジョブを投入してみましょう。<br>
AML Studio の「ジョブ(Job)」メニューから進捗と結果を確認します。

In [ ]:
!az ml job create --file 02_train_on_remote.yml

ジョブが完了するまでお待ちください。
AML studio UI("Jobs "ペイン参照)または以下の CLI コマンドで進捗と結果を表示することができます。<br>
nifty_tree_cftvbh77f7 を**生成したジョブ名で置き換えてください**。ジョブ名については、上記の出力を参照してください


In [ ]:
job_name = "nifty_tree_cftvbh77f7"

In [ ]:
!az ml job show --name $job_name

### Step 4 : 結果のダウンロードと評価

生成されたモデルをローカルコンピュータで確認してみましょう。

[Azure ML studio UI](https://ml.azure.com/) にアクセスします。<br>
すると、models ディレクトリに保存されたモデルが表示されます。


![Saved Outputs](../images/010.png)

以下の ``az ml job download`` コマンドを実行すると、ログと出力がローカルにダウンロードされます。 <br>
ログは ``artifacts/xxx_logs`` に、出力は ``artifacts/models`` に保存されます。


In [ ]:
!az ml job download --name $job_name

ダウンロードしたモデルをローカルで確認します。

In [ ]:
import mlflow.sklearn
import warnings
warnings.filterwarnings('ignore')

model_path = "./artifacts/models"
model = mlflow.sklearn.load_model(model_path)

# sample query
input = [[0.0380759064334241,0.0506801187398187,0.0616962065186885,0.0218723549949558,-0.0442234984244464,-0.0348207628376986,-0.0434008456520269,-0.00259226199818282,0.0199084208763183,-0.0176461251598052]]

predictions = model.predict(input)
log_txt = 'Predictions:' + str(predictions)
print (log_txt)


### Step 5 : モデルの登録

モデルを AML モデルレジストリにアップロード（登録）します。<br>
登録は CLI v2 からも可能ですが、今回は一例として MLFlow のインターフェースを利用しています。
MLFlow の `register_model` を使用するとジョブ名からモデルを登録できます。

In [ ]:
import mlflow
model_uri = "runs:/{}/models".format(job_name)
mv = mlflow.register_model(model_uri, "diabetes_model_oh4ml")
mv

### Step 6 : AML モデルレジストリに登録されたモデルをロード
MLFlow を利用して AML モデルレジストリに登録されたモデルを取得します。<br>
モデルはすでにバージョン管理されており、モデルを呼び出すにはモデル名とモデルバージョン番号が必要になります。

In [ ]:
from mlflow.tracking import MlflowClient
model_name = "diabetes_model_oh4ml"
model_stage = "None"

# Use MLFlow to get the latest model
client = MlflowClient()
model_version = client.get_latest_versions(model_name, stages=[model_stage])[0].version
model_uri = "models:/{model_name}/{model_version}".format(model_name=model_name, model_version=model_version)
model = mlflow.sklearn.load_model(model_uri)
print(model)

# sample query
input = [[0.0380759064334241,0.0506801187398187,0.0616962065186885,0.0218723549949558,-0.0442234984244464,-0.0348207628376986,-0.0434008456520269,-0.00259226199818282,0.0199084208763183,-0.0176461251598052]]

predictions = model.predict(input)
log_txt = 'Predictions:' + str(predictions)
print (log_txt)

### Step 7 : AML コンピュートクラスターの削除

AML コンピュートが非アクティブになると、ノードは自動的に終了するので、**コストを節約するために削除する必要はありません**。
しかし、もしクリーンアップしたい場合は、以下のように実行してください。

In [ ]:
"""
!az ml compute delete --name demo-cpucluster1 \
  --yes
"""